In [1]:
#the below two lines of code are only for display purposes when using jupyter notebook. If using another IDE, you can delete the next two lines
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import load

df = load.Main_Dataset()

sets = df.sec_ichars
sets = [val if val!='[]' else val for val in sets]
te = TransactionEncoder()
te_ary = te.fit(sets).transform(sets)
fi_df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(fi_df, min_support=0.01, use_colnames=True)
# alternatively:
# frequent_itemsets = apriori(asc_df, min_support=0.01, use_colnames=True)
# frequent_itemsets = fpmax(asc_df, min_support=0.01, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False).reset_index(drop=True)
frequent_itemsets

Filtering process eliminated 0.0% of the initial data 



,support,itemsets
0,0.128769,(Possession (gun(s) found during commission of...
1,0.082290,(Armed robbery with injury/death and/or eviden...
2,0.079344,(Brandishing/flourishing/open carry/lost/found)
3,0.075064,(ATF/LE Confiscation/Raid/Arrest)
4,0.075051,(Officer Involved Incident)
...,...,...
63,0.011169,(Kidnapping/abductions/hostage)
64,0.011140,(Possession (gun(s) found during commission of...
65,0.011052,"(Defensive Use, Defensive Use - Crime occurs, ..."
66,0.011052,"(Defensive Use - Crime occurs, victim shoots s..."


In [2]:
cico_per_state = cico_dat.groupby(['state_abr']).count()
cico_per_state.reset_index(inplace = True)
cico_per_state = cico_per_state.drop(cico_per_state.columns.difference(['state_abr', 'city_or_county']), 1)
cico_per_state.columns = ['state_abr', 'n_cico_instate']

n_rik_per_state = cico_dat.groupby(['state_abr'])[['rpt_qnt', 'inj_qnt', 'kill_qnt']].apply(sum)
n_rik_per_state.columns = ['total_rpt_qnt', 'total_inj_qnt', 'total_kill_qnt']
n_rik_per_state.reset_index(inplace = True)
state_profiles = pd.merge(n_rik_per_state, cico_per_state, on='state_abr')
state_profiles['rpt_qnt'] = state_profiles['total_rpt_qnt']/state_profiles['n_cico_instate']
state_profiles['inj_qnt'] = state_profiles['total_inj_qnt']/state_profiles['n_cico_instate']
state_profiles['kill_qnt'] = state_profiles['total_kill_qnt']/state_profiles['n_cico_instate']
state_profiles.to_csv('state_profiles.csv')

In [6]:
from mlxtend.frequent_patterns import association_rules
asc_tbl = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.0)
asc_tbl = asc_tbl.sort_values(by='lift', ascending=False).reset_index(drop=True)
asc_tbl.to_csv('asc_tbl.csv',index=False)
asc_tbl

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Defensive Use - Crime occurs, victim shoots s...",(Defensive Use),0.011052,0.030837,0.011052,1.000000,32.428224,0.010712,inf
1,(Defensive Use - Victim stops crime),(Defensive Use),0.011399,0.030837,0.011399,1.000000,32.428224,0.011047,inf
2,(Defensive Use),(Defensive Use - Victim stops crime),0.030837,0.011399,0.011399,0.369639,32.428224,0.011047,1.568309
3,(Defensive Use),"(Defensive Use - Crime occurs, victim shoots s...",0.030837,0.011052,0.011052,0.358409,32.428224,0.010712,1.541399
4,(Accidental/Negligent Discharge),"(Accidental Shooting - Injury, Accidental Shoo...",0.026794,0.022388,0.017607,0.657116,29.350658,0.017007,2.851145
...,...,...,...,...,...,...,...,...,...
95,(Officer Involved Incident),(Brandishing/flourishing/open carry/lost/found),0.075051,0.079344,0.012388,0.165054,2.080232,0.006433,1.102654
96,(Possession (gun(s) found during commission of...,(Brandishing/flourishing/open carry/lost/found),0.128769,0.079344,0.012179,0.094579,1.192011,0.001962,1.016826
97,(Brandishing/flourishing/open carry/lost/found),(Possession (gun(s) found during commission of...,0.079344,0.128769,0.012179,0.153494,1.192011,0.001962,1.029208
98,(Possession (gun(s) found during commission of...,(Officer Involved Incident),0.128769,0.075051,0.011140,0.086511,1.152701,0.001476,1.012546
